# What If Fantasy Football Had Been Around When The NFL Formed?

https://developer.sportradar.com/files/indexFootball.html?python#nfl-official-api-v2

This endpoint retrieves the Seasonal Statistics for a season.

https://api.sportradar.us/nfl-{access_level}{version}/seasontd/{year}/{nfl_season}/teams/{team_id}/statistics.{format}?api_key={your_api_key}

Replace placeholders with the following query parameters:

Parameter                 Description
access_level	          Defines the access level of your API key as Production (o), Trial (ot), or Simulation (sim). 
                          For more information on simulations, see NFL Official API v2 Simulations.
version	                  Version number of the API you are accessing (current version: 2).
year	                  Year in 4 digit format (YYYY).
nfl_season	              Preseason (PRE), Regular Season (REG), or Post-Season (PST).
team_id	                  ID of a given team.
format	                  xml or json.
your_api_key	          Your API key.


In [2]:
# Importing Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
import os
import json
import numpy as np
import seaborn as sb
import time

In [3]:
api_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath('__file__')))))
file_name = os.path.join(api_dir, "api_keys.json")
data = json.load(open(file_name))

c_key = data['sportradar_nfl_classic_key']
o_key = data['sportradar_nfl_official_key']

In [4]:
# Setting up URL and API call for team_id
url = "http://api.sportradar.us/nfl-ot2/seasontd/"
season = "REG"
year = "2017"
output_format = "json"
team_id = "33405046-04ee-4058-a950-d606f8c30852"

query_url = url + year + "/" + season + "/teams/" + team_id + "/statistics." + output_format + "?api_key=" + o_key
nfl_response = requests.get(query_url)
nfl_json = nfl_response.json()

print(json.dumps(nfl_json, indent=4))

{
    "id": "33405046-04ee-4058-a950-d606f8c30852",
    "name": "Vikings",
    "market": "Minnesota",
    "alias": "MIN",
    "season": {
        "id": "3cf0e8ec-048d-4872-a706-b248c198831c",
        "year": 2017,
        "type": "REG",
        "name": "REG"
    },
    "record": {
        "games_played": 16,
        "touchdowns": {
            "pass": 25,
            "rush": 15,
            "total_return": 1,
            "total": 41,
            "fumble_return": 0,
            "int_return": 1,
            "kick_return": 0,
            "punt_return": 0,
            "other": 0
        },
        "rushing": {
            "avg_yards": 3.906,
            "attempts": 501,
            "touchdowns": 15,
            "tlost": 55,
            "tlost_yards": -135,
            "yards": 1957,
            "longest": 58,
            "longest_touchdown": 58,
            "redzone_attempts": 25,
            "broken_tackles": 33,
            "kneel_downs": 21,
            "scrambles": 17,
            "yar

# Team ID List

In [7]:
team_id_url = "http://api.sportradar.us/nfl-ot2/league/hierarchy.json?api_key="

team_id_response = requests.get(team_id_url + o_key)
team_id_json = team_id_response.json()

print(json.dumps(team_id_json, indent=4))

{
    "league": {
        "id": "3c6d318a-6164-4290-9bbc-bf9bb21cc4b8",
        "name": "National Football League",
        "alias": "NFL"
    },
    "conferences": [
        {
            "id": "1bdefe12-6cb2-4d6a-b208-b04602ae79c3",
            "name": "AFC",
            "alias": "AFC",
            "divisions": [
                {
                    "id": "b95cd27d-d631-4fe1-bc05-0ae47fc0b14b",
                    "name": "AFC East",
                    "alias": "AFC_EAST",
                    "teams": [
                        {
                            "id": "768c92aa-75ff-4a43-bcc0-f2798c2e1724",
                            "name": "Bills",
                            "market": "Buffalo",
                            "alias": "BUF",
                            "references": [
                                {
                                    "id": "BUF",
                                    "origin": "gsis"
                                }
                            ],
    

In [8]:
# Setting up sentiments
conf_count = 0
conferences = []
divisions = []
teams = []
team_id = []
venue_name = []
venue_city = []
venue_state = []
venue_capacity = []
venue_surface = []
venue_roof_type = []

# Starting conference gathering loop
for conference in team_id_json['conferences']:

    # Declaring division counter inside so that it resets for each iteration
    div_count = 0
    
    # Starting division gathering loop
    for division in team_id_json['conferences'][conf_count]:
        
        # Declaring teams counter inside so that it resets for each iteration
        team_count = 0
        
        # Starting team gathering loop
        for team in team_id_json['conferences'][conf_count]['divisions'][div_count]:

            # Creating variables for easier handling when appending
            temp_name = team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['name']
            temp_market = team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['market']
            
            # Appending conferences, divisions, team name and team IDs to lists
            conferences.append(team_id_json['conferences'][conf_count]['name'])
            divisions.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['name'])
            teams.append((str(temp_market) + " " + str(temp_name)))
            team_id.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['id'])
            venue_name.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['name'])
            venue_city.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['city'])
            venue_state.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['state'])
            venue_capacity.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['capacity'])
            venue_surface.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['surface'])
            venue_roof_type.append(team_id_json['conferences'][conf_count]['divisions'][div_count]['teams'][team_count]['venue']['roof_type'])
            
            # Incrementing teams counter
            team_count += 1
            
        # Incrementing division counter
        div_count += 1
    
    # Incrementing conference counter
    conf_count += 1

In [22]:
teams_df = pd.DataFrame(list(zip(teams, conferences, divisions, venue_name, venue_city, venue_state, venue_capacity, \
                                 venue_surface, venue_roof_type, team_id)), \
                                 columns=['Team', 'Conference', 'Division', 'Venue Name', 'Venue City', \
                                          'Venue State', 'Venue Capacity', 'Playing Surface', 'Stadium Type', 'Team ID'])
teams_df.head(5)

,Team,Conference,Division,Venue Name,Venue City,Venue State,Venue Capacity,Playing Surface,Stadium Type,Team ID
0,Buffalo Bills,AFC,AFC East,New Era Field,Orchard Park,NY,73079,artificial,outdoor,768c92aa-75ff-4a43-bcc0-f2798c2e1724
1,Miami Dolphins,AFC,AFC East,Hard Rock Stadium,Miami Gardens,FL,76100,turf,outdoor,4809ecb0-abd3-451d-9c4a-92a90b83ca06
2,New York Jets,AFC,AFC East,MetLife Stadium,East Rutherford,NJ,82500,artificial,outdoor,5fee86ae-74ab-4bdd-8416-42a9dd9964f3
3,New England Patriots,AFC,AFC East,Gillette Stadium,Foxborough,MA,68756,artificial,outdoor,97354895-8c77-4fd4-a860-32e62ea7382a
4,Cincinnati Bengals,AFC,AFC North,Paul Brown Stadium,Cincinnati,OH,65535,artificial,outdoor,ad4ae08f-d808-42d5-a1e6-e9bc4e34d123
5,Cleveland Browns,AFC,AFC North,FirstEnergy Stadium,Cleveland,OH,71516,turf,outdoor,d5a2eb42-8065-4174-ab79-0a6fa820e35e
6,Baltimore Ravens,AFC,AFC North,M&T Bank Stadium,Baltimore,MD,71008,turf,outdoor,ebd87119-b331-4469-9ea6-d51fe3ce2f1c
7,Pittsburgh Steelers,AFC,AFC North,Heinz Field,Pittsburgh,PA,65050,turf,outdoor,cb2f9f1f-ac67-424e-9e72-1475cb0ed398
8,Indianapolis Colts,AFC,AFC South,Lucas Oil Stadium,Indianapolis,IN,67000,artificial,retractable_dome,82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9
9,Jacksonville Jaguars,AFC,AFC South,EverBank Field,Jacksonville,FL,67246,turf,outdoor,f7ddd7fa-0bae-4f90-bc8e-669e4d6cf2de


In [25]:
# teams_df.groupby('Stadium Type').mean()
teams_df['Team ID'][0]

'768c92aa-75ff-4a43-bcc0-f2798c2e1724'

In [11]:
teams_df.to_csv('teams_df.csv')

In [89]:
all_teams_dict = {}
#loop through each teams data
for x in range(0,2):
# change range above for desired # of teams, to get full list use
# for x in range(0,len(teams_df)):
    team_id = teams_df['Team ID'][x]
    team_dict = {}
    
#     loop throug each years data
    for y in range(0, 2):
#       change range above for desired of years, where 0,17 is the max range (2000 - 2017)
        year = y + 2000
        team_season_url = 'http://api.sportradar.us/nfl-ot2/seasontd/' + str(year) + '/reg/teams/' + team_id + '/statistics.json?api_key=' + o_key
        team_season_response = requests.get(team_season_url)
        time.sleep(1)
        response = team_season_response.json()
        team_dict[y] = response
    all_teams_dict[x] = team_dict

In [96]:
all_teams_dict[0][0]['players']
# first [0] indicates team
# second [0] indicates year

[{'games_played': 1,
  'games_started': 0,
  'id': '05b0c05e-ec5b-4c41-9ff6-55ffcbdab955',
  'jersey': '10',
  'name': 'Alex Van Pelt',
  'passing': {'air_yards': 0,
   'attempts': 8,
   'avg_pocket_time': 0.0,
   'avg_yards': 8.375,
   'blitzes': 0,
   'cmp_pct': 50.0,
   'completions': 4,
   'defended_passes': 0,
   'dropped_passes': 0,
   'gross_yards': 67,
   'hurries': 0,
   'interceptions': 0,
   'knockdowns': 0,
   'longest': 36,
   'longest_touchdown': 0,
   'pocket_time': 0.0,
   'poor_throws': 0,
   'rating': 78.6,
   'redzone_attempts': 0,
   'sack_yards': 0,
   'sacks': 0,
   'spikes': 0,
   'throw_aways': 0,
   'touchdowns': 0,
   'yards': 67},
  'position': 'QB',
  'reference': '00-0016838'},
 {'games_played': 16,
  'games_started': 3,
  'id': '1454f252-2a45-4a4d-bce6-c9a4958514cb',
  'jersey': '86',
  'name': 'Jeremy McDaniel',
  'position': 'WR',
  'receiving': {'air_yards': 697,
   'avg_yards': 16.209,
   'broken_tackles': 0,
   'catchable_passes': 0,
   'dropped_passe

<Response [403]>

In [61]:
team_dict[2001]

{'_comment': 'Generation started @ 2018-02-02 02:02:19 +0000 ended @ 2018-02-02 02:02:19 +0000',
 'alias': 'MIA',
 'id': '4809ecb0-abd3-451d-9c4a-92a90b83ca06',
 'market': 'Miami',
 'name': 'Dolphins',
 'opponents': {'defense': {'assists': 275,
   'blitzes': 0,
   'combined': 904,
   'def_comps': 0,
   'def_targets': 0,
   'forced_fumbles': 17,
   'fumble_recoveries': 15,
   'hurries': 0,
   'interceptions': 19,
   'knockdowns': 0,
   'misc_assists': 0,
   'misc_forced_fumbles': 1,
   'misc_fumble_recoveries': 16,
   'misc_tackles': 17,
   'missed_tackles': 0,
   'passes_defended': 70,
   'qb_hits': 30,
   'sack_yards': 178.0,
   'sacks': 27.0,
   'safeties': 0,
   'sp_assists': 22,
   'sp_blocks': 1,
   'sp_forced_fumbles': 5,
   'sp_fumble_recoveries': 4,
   'sp_tackles': 89,
   'tackles': 629,
   'tloss': 78.0,
   'tloss_yards': 246.0},
  'efficiency': {'fourthdown': {'attempts': 15, 'pct': 40.0, 'successes': 6},
   'goaltogo': {'attempts': 19, 'pct': 68.421, 'successes': 13},
   'r

In [1]:
#team_dict[y]

# Setting up Fantasy endpoints

|Positions To Gather            |
|:-----------------------------:|
|Position | QB, HB, FB, WR, TE  |

|General Stats|   Endpoint      |
|:------------|:----------------|
|games_played | ['games_played']|
|player_id    | ['id']          |
|player_name  | ['name']        |
|position     | ['position']    |

| Offense           | Endpoint                     | Standard | PPR     |
|:------------------|:-----------------------------| -------: | -------:|
| passing_yards     | ['passing']['yards']         | .04 pts  |.04 pts  |
| passing_tds       | ['passing']['touchdowns']    |   4 pts  |  4 pts  |
| interceptions     | ['passing']['interceptions'] |  -2 pts  | -2 pts  |
| rushing_yards     | ['rushing']['yards']         |  .1 pts  | .1 pts  |
| rushing_tds       | ['rushing']['touchdowns']    |   6 pts  |  6 pts  |
| fumbles           | ['fumbles']['fumbles']       |  -2 pts  | -2 pts  | 
| receiving_yards   | ['receiving']['yards']       |  .1 pts  | .1 pts  |
| receiving_tds     | ['receiving']['touchdowns']  |   6 pts  |  6 pts  |
| receptions        | ['receiving']['receptions']  |   0 pts  |  1 pts  |

| Misc stats        | Endpoint                       | All Formats |
|:------------------|:-------------------------------|------------:|
|punt_td            | ['kick_returns']['touchdowns'] |  6 pts      |
|kick_td            | ['punt_returns']['touchdowns'] |  6 pts      |

In [ ]:
test_df = all_teams_dict[0][0]['players']